# DL 复习

## 目录

## Lect 2 线性回归，二分类回归，损失函数

### 线性回归

- 在线性回归中， 如果假定误差是正态，极大似然可以推导出OLS，即MSE准则
- 若假定误差是Laplace分布，极大似然可以推导出MAE准则

### 线性二分类

$$Y|x \sim Bernoulli(\rho(\beta'x))$$

- $\rho(\beta'x)$把$\beta'x$映射到$(0,1)$，表示$Y=1$的概率
- 当$\rho(x)=e^x/(e^x+1)$时，称为logistic 回归

***Lositics 回归***
- 损失函数
  $$\begin{aligned} 
  L(\beta)&=n^{-1}\sum_{i=1}^n l(\rho(\beta'x_i),y_i) 
  \\
  &= n^{-1}\sum_{i=1}^n \left[ y_i\log \rho(\beta'x_i) + (1-y_i)\log(1-\rho(\beta'x_i)) \right]
  \end{aligned}$$


In [ ]:
# logistics MLE loss function

def loss_fn_logistic(bhat, x, y):
    rhohat = sigmoid(torch.matmul(x,bhat))
    loss = -torch.sum(y*torch.log(rhohat) + (1-y) * torch.log(1-rhohat))/y.shape[0]
    return loss

### 其他损失函数

- KL 散度
- 最小化KL散度等价于MLE

## Lect 3 Pytorch 基础、线性多分类

### PyTorch 基本操作

基本数据操作

In [ ]:
import numpy as np
import torch

# 随机数种子
np.random.seed(123456)
torch.manual_seed(123456)

# 创建向量/矩阵
vec = torch.tensor([1.0, 2.0, 5.0])
print(vec)
mat = torch.tensor([[1.0, 2.0, 2.0], [3.0, 5.0, 4.5]])
print(mat)

# 特殊向量
torch.ones(3, 2)
torch.zeros(5)
torch.linspace(3, 10, steps=5)
torch.randn(2, 3)

# 矩阵形状与变形
print(vec.shape)
print(mat.shape)
n = mat.shape[0]
p = mat.shape[1]

print(vec.view(3, 1))
print(mat.view(3, 2))

# 汇总
print(mat)
print(torch.sum(mat, dim=0))
print(torch.sum(mat, dim=1))

# 矩阵计算
torch.matmul(torch.t(mat), mat)
mat.t()

统计分布

In [ ]:
import torch.distributions as D
import math

# 正态分布【注意，scale传入标准差】
## 生成分布形式
norm = D.Normal(loc=torch.tensor([1.0]), scale=torch.tensor([math.sqrt(3.0)]))
## 求正态分布某点对数密度函数
norm.log_prob(torch.tensor([1.0, 2.0, 3.0]))
## 生成正态随机数
norm.sample(sample_shape=(5,))

# Bernoulli分布
## 可以同时指定多组Bernoulli分布
bern = D.Bernoulli(probs=torch.tensor([0.1, 0.5, 0.9]))
bern.sample()

其他常见的操作可以参考[官方教程](https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html)，完整的函数列表可以查看[官方 API 文档](https://pytorch.org/docs/stable/torch.html)。

### 线性多分类

$$Y|x \sim Multinomial(\rho(Wx))$$

- $x_i \in \mathbb{R}^p$: 每个obs的数据包括p个特征
- $y_i \in \mathbb{R}^k$: 每个obs的数据包括k个类别，且此处是one-hot编码，即$y_i$只有一个元素为1，其余为0
- $W \in \mathbb{R}^{k \times p}$: 负责将$p$维的特征映射到$k$维的类别
- $\rho(Wx)$: 将$Wx$映射到$(0,1)^k$，表示每个类别的概率

***Softmax 回归***
- 当取$\rho(Wx)=\frac{e^{Wx}}{\sum_{j=1}^k e^{Wx_j}}$时，称为Softmax回归
- 多项函数的对数似然为：$l(p;y) = \sum_{j=1}^k y_j \log p_j$, 其中$p = (p_1,\dots,p_k) = \rho(Wx)$
- 损失函数为：$L(W) = n^{-1} \sum_{i=1}^n l(\rho(Wx_i);Y_i)$

In [ ]:
def loss_fn_softmax(w, x, y):

    rho = torch.softmax(torch.matmul(x,w.t()),dim=1)
    l = torch.sum(torch.log(rho)*y)/y.shape[0]
    return - l

## Lect4 前馈神经网络(数值稳定)、反向传播

### 前馈神经网络

***激活函数***

详见后面

***数值稳定***

[Sigmoid]

$\exp(x)$当$x$过大时发生溢出。可以上下同除以$\exp(x)$得到：
$$ \sigma(x) = \frac{1}{1+\exp(-x)}$$

In [ ]:
def sigmoid2(x):
    sig  = 1 / ( 1 + torch.exp(-x) )
    return sig
# PyTorch 自带函数
print(torch.sigmoid(x))

[tanh]

当 $x>>0$时，分子$e^x$会发生溢出。可以上下同乘以$\exp(-x)$得到：
$$ \sigma(x) = \frac{1-\exp(-2x)}{1+\exp(-2x)}$$
当 $x<<0$时，分母$e^{-x}$会发生溢出。可以上下同乘以$\exp(x)$得到：
$$ \sigma(x) = \frac{\exp(2x)-1}{\exp(2x)+1}$$

In [ ]:
def tanh2(x):
    exp = torch.exp( -2 * torch.abs(x) ) # 此行代码灵感借鉴自ds2023s中hw4的参考答案
    th = torch.where( x>0 , (1-exp)/(1+exp) , (exp-1)/(exp+1) ) 
    return th
# PyTorch 自带函数
print(torch.tanh(x))

[softplus]

当x>>0时，$e^x$会发生溢出。可以有如下变形：
$$ \mathrm{softplus}(x) = \log(1+e^x) = x + \log(1+e^{-x})$$

In [ ]:
def softplus2(x):
    lg = torch.log( torch.exp( -torch.abs(x)) + 1)
    sp = torch.where(x>0, x + lg, lg )
    return sp
# PyTorch 自带函数
print(torch.nn.functional.softplus(x))

[log sigmoid (logistics loss function)]

$$\log\hat\rho = \log\frac{e^{X\beta}}{1+e^{X\beta}} = X\beta - \log(1+e^{X\beta}) = X\beta  - \mathrm{softplus}(X\beta)$$

In [ ]:
def loss_fn_logistic(bhat, x, y):
    xbhat = torch.matmul(x,bhat)
    logrho1 = xbhat - softplus_fn(xbhat)
    logrho2 = - softplus_fn (xbhat)
    loss = -torch.sum( y*logrho1 + (1-y)* logrho2 ) / y.shape[0]
    return loss  

***前馈网络***

其中的$x_1,\dots,x_p$为输入层，分别输入的是一个obs中的不同的features；不同的obs相当于一次丢进这个网络中

每一层的网络都可以看做是一组features，是activated的状态；经过线性映射W^(l)被映射到下一层，归纳成了另外$M_l$个元素，而经过激活函数激活后，这些元素就是下一层的输入

![](https://michael-1313341240.cos.ap-shanghai.myqcloud.com/202312222351646.png)

***通用近似定理***

两层的网络几乎可以拟合任意函数